In [18]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn
import matplotlib.pyplot as plt
import random
import pandas as pd


class Patient:
    def __init__(self, id):
        self.id = id
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None
        self.room = None
        self.real_service_time = None

class Room:
    def __init__(self, id, env):
        self.id = id
        self.patients_served = 0
        self.no_show = 0
        self.queue = []
        self.resource = simpy.Resource(env, capacity=1)

class Clinic:
    def __init__(self, number_of_rooms, planned_service_time, no_show=0.2, seed=None, sim_time=480):
        self.curr_patient_id = 1
        self.env = simpy.Environment()
        self.planned_service_time = planned_service_time
        self.no_show = no_show
        self.seed = seed
        self.list_rooms = [Room(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.processed_patients = []
        self.sim_time = sim_time

        if self.seed:
            random.seed(self.seed)

    def readable_time(self):
        hours = 8 + int(self.env.now // 60)
        minutes = int(self.env.now % 60)
        if len(str(minutes)) == 1:
            return f"{hours}:0{minutes}"
        return f"{hours}:{minutes}"

    def generate_patients(self, room):

        def randomized_service_time(mean=15, std=2):
            return max(np.random.normal(mean, std), 1)

        def time_between_new_patients():
            return self.planned_service_time

        while True:
            if self.env.now > self.sim_time - self.planned_service_time:  # Pacjenci nie przychodzą przed {service_time} zamknięciem
                break

            patient = Patient(id=f"{room.id}.{self.curr_patient_id}")
            patient.room = room.id
            patient.real_service_time = randomized_service_time()

            if random.random() < self.no_show:  #pacjent nie przyszedl
                print(f'Czas {self.readable_time()}: Pacjent {patient.id} nie pojawił w gabinecie {room.id} w ciągu {self.planned_service_time} minut')
                room.no_show += 1

            else:  #pacjent przyszedl
                patient.arrival_time = self.env.now
                print(f"Czas {self.readable_time()}: Pacjent {patient.id} przybył do kliniki")
                self.env.process(self.serve_patient(patient, room))

            self.curr_patient_id += 1
            yield self.env.timeout(time_between_new_patients())

    def serve_patient(self, patient, room):
        room.queue.append(patient.id)

        with room.resource.request() as request:
            yield request
            patient.service_start_time = self.env.now
            room.queue.remove(patient.id)

            print(f"Czas {self.readable_time()}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")

            yield self.env.timeout(patient.real_service_time)

            patient.service_end_time = self.env.now
            room.patients_served += 1
            print(f"Czas {self.readable_time()}: Pacjent {patient.id} wychodzi z gabinetu {room.id}")
            self.processed_patients.append(patient)

    def run(self, day=1):
        for room in self.list_rooms:
            self.env.process(self.generate_patients(room))
        self.env.run(until=self.sim_time + 0.01)
        df = pd.DataFrame([
            {
                "day": day,
                "id": p.id,
                "room": p.room,
                "arrival_time": p.arrival_time,
                "service_start_time": p.service_start_time,
                "service_end_time": p.service_end_time,
                "service_time": p.service_end_time - p.service_start_time,
                "waiting_time": p.service_start_time - p.arrival_time
            }
            for p in self.processed_patients
        ])
        df = df.set_index("id")
        return df

    def run_multiple_times(self, n):
        sum_no_show = 0
        sum_patients_served = 0
        for i in range(1, n + 1):
            print(f"-----------{i}-----------")
            self.env = simpy.Environment()
            self.curr_patient_id = 1
            self.list_rooms = [Room(id=i + 1, env=self.env) for i in range(len(self.list_rooms))]
            self.processed_patients = []
            single_df = self.run(i)
            for room in self.list_rooms:
                sum_no_show += room.no_show
                sum_patients_served += room.patients_served
            multiple_df = pd.concat([multiple_df, single_df], ignore_index=True) if i > 1 else single_df
        return multiple_df, sum_patients_served, sum_no_show

In [21]:
clinic1 = Clinic(number_of_rooms=2, planned_service_time=15, sim_time=120)
df = clinic1.run()

Czas 8:00: Pacjent 1.1 nie pojawił w gabinecie 1 w ciągu 15 minut
Czas 8:00: Pacjent 2.2 przybył do kliniki
Czas 8:00: Pacjent 2.2 wchodzi do gabinetu 2 
Czas 8:13: Pacjent 2.2 wychodzi z gabinetu 2
Czas 8:15: Pacjent 1.3 przybył do kliniki
Czas 8:15: Pacjent 2.4 przybył do kliniki
Czas 8:15: Pacjent 1.3 wchodzi do gabinetu 1 
Czas 8:15: Pacjent 2.4 wchodzi do gabinetu 2 
Czas 8:30: Pacjent 1.5 nie pojawił w gabinecie 1 w ciągu 15 minut
Czas 8:30: Pacjent 2.6 przybył do kliniki
Czas 8:30: Pacjent 1.3 wychodzi z gabinetu 1
Czas 8:32: Pacjent 2.4 wychodzi z gabinetu 2
Czas 8:32: Pacjent 2.6 wchodzi do gabinetu 2 
Czas 8:45: Pacjent 1.7 przybył do kliniki
Czas 8:45: Pacjent 2.8 przybył do kliniki
Czas 8:45: Pacjent 1.7 wchodzi do gabinetu 1 
Czas 8:48: Pacjent 2.6 wychodzi z gabinetu 2
Czas 8:48: Pacjent 2.8 wchodzi do gabinetu 2 
Czas 9:00: Pacjent 1.9 przybył do kliniki
Czas 9:00: Pacjent 2.10 przybył do kliniki
Czas 9:03: Pacjent 1.7 wychodzi z gabinetu 1
Czas 9:03: Pacjent 1.9 wchodzi

display(df)